In [ ]:
from itertools import product
import math
import numpy as np
import pandas as pd
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression
import scipy
from scipy.optimize import minimize
from scipy.special import expit
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
class World:
  def __init__(self, slot_count, level_count_per_slot, additive=True, input_marginals=None):
    self.slot_count = slot_count
    self.level_count_per_slot = level_count_per_slot
    self.additive = additive
    
    self.marginals_low = 0.05
    self.marginals_high = 0.15
    
    self.arm_count = self.level_count_per_slot ** self.slot_count
    
    self.binary_feature_vector_per_arm = list(map(np.hstack, list(product(
      *([pd.get_dummies(range(level_count_per_slot)).values] * slot_count)))))
    assert(len(self.binary_feature_vector_per_arm) == self.arm_count)
    
    self.slot_level_map_per_arm = []
    for v in self.binary_feature_vector_per_arm:
      indices, = np.where(v > 0)
      indices = indices % level_count_per_slot
      self.slot_level_map_per_arm.append(tuple(indices))
      
    self.slot_level_map_to_arm = dict(zip(self.slot_level_map_per_arm, range(self.arm_count)))
    
    if input_marginals is None:
      self.marginals_per_slot_per_level = np.random.uniform(low=self.marginals_low, high=self.marginals_high,
                                                            size=(self.slot_count, self.level_count_per_slot))
    else:
      assert(input_marginals.shape == (self.slot_count, self.level_count_per_slot))
      self.marginals_per_slot_per_level = deepcopy(input_marginals)

    self.expected_reward_probabilities = np.zeros(self.arm_count)
    for arm in range(self.arm_count):
      slot_to_level_map = self.slot_level_map_per_arm[arm]
      for slot in range(self.slot_count):
        level = slot_to_level_map[slot]
        self.expected_reward_probabilities[arm] += self.marginals_per_slot_per_level[slot, level]
        if self.additive == False:
          for slot2 in range(self.slot_count):
            level2 = slot_to_level_map[slot2]
            if np.random.random() < 1:
              coefficient = np.random.randint(low=10, high=20)
              self.expected_reward_probabilities[arm] += (self.marginals_per_slot_per_level[slot, level] * 
                                                           self.marginals_per_slot_per_level[slot2, level2])
    if self.additive == False:
      self.expected_reward_probabilities = expit(2*(self.expected_reward_probabilities-1))
    


In [ ]:
# Agent Base Class

class MABAgent:
  def __init__(self, world, horizon):
    self.world = world
    self.horizon = horizon
     
    # Oracle success probabilities.
    self.oracle_success_probabilities = self.world.expected_reward_probabilities
     
    # Per period regret of the policy throughout the horizon.
    self.regret_per_period = np.zeros(self.horizon)
    # Per period reward of the policy throughout the horizon.
    self.reward_per_period = np.zeros(self.horizon)
     
    # Oracle success probability of optimal arm.
    self.optimal_arm = np.argmax(self.oracle_success_probabilities)
    self.optimal_arm_oracle_success_probability = self.oracle_success_probabilities[self.optimal_arm]
     
    # Cache observations for causal inference.
    # We cache the time-stamp of the observation, the selected arm and the observed reward.
    self.observations_cache = np.zeros((self.horizon, 3))
         
  def run(self):
    for t in range(0, self.horizon):
      # Choose an arm to play.
      selected_arm = self.act(t)

      # Observe the reward of the selected arm.
      oracle_success_probability_of_selected_arm = self.oracle_success_probabilities[selected_arm]
      reward = np.random.binomial(1, oracle_success_probability_of_selected_arm)

      # Update the posterior distribution of the selected arm.
      self.update(selected_arm, reward, t)

      # Save the reward and the expected regret.
      self.reward_per_period[t] = reward
      self.regret_per_period[t] = np.max(self.oracle_success_probabilities) - oracle_success_probability_of_selected_arm

      # Cache the observation.
      self.observations_cache[t, 0] = t
      self.observations_cache[t, 1] = selected_arm
      self.observations_cache[t, 2] = reward
                                                         
  def act(self, t):
    pass
   
  def update(self, selected_arm, reward, t):
    pass
 

In [ ]:
# Independent Bernoulli Arms Agent

class IndependentBernoulliArmsTSAgent(MABAgent):
  def __init__(self, world, horizon):
    MABAgent.__init__(self, world, horizon)
   
    # The prior distribution of the mean rewards of the arms is a Beta distribution, 
    # with alpha = 1 and beta = 1 (uniform).
    self.alphas = np.ones(self.world.arm_count)
    self.betas = np.ones(self.world.arm_count)
    
    self.posterior_updating_frequency = 20
    self.running_alphas = np.ones(self.world.arm_count)
    self.running_betas = np.ones(self.world.arm_count)
    
   
  def act(self, t):
    estimated_reward = np.zeros(self.world.arm_count)
    for arm in range(0, self.world.arm_count):
      alpha = self.alphas[arm]
      beta = self.betas[arm]
      estimated_reward[arm] = np.random.beta(alpha, beta)
 
    if np.all(estimated_reward == estimated_reward[0]):
        return np.random.randint(self.world.arm_count)
    else:
        return np.argmax(estimated_reward)
     
  def update(self, selected_arm, reward, t):
    if reward > 0:
      self.running_alphas[selected_arm] += 1
    else:
      self.running_betas[selected_arm] += 1
    if t > 0 and t % self.posterior_updating_frequency == 0:
      self.alphas = deepcopy(self.running_alphas)
      self.betas = deepcopy(self.running_betas)

In [ ]:
# Logistic Regression Agents

class LogisticRegressionAgent(MABAgent):
  def __init__(self, world, horizon, regularization_parameter):
    MABAgent.__init__(self, world, horizon)
     
    self.regularization_parameter = regularization_parameter
     
    # Batch caches
    self.batch_X = []
    self.batch_y = []
     
    # Frequency of posterior updating.
    self.posterior_updating_frequency = 20
     
    # There are dummy variables in the logistic function, representing the features of the arms.
    # The prior for the means in zero and for the variances 1 / lambda, where lambda is a regularization parameter.
    self.mu = np.zeros(self.world.slot_count * self.world.level_count_per_slot)
    self.sigma_inverse = np.identity(self.world.slot_count * self.world.level_count_per_slot) * self.regularization_parameter
                           
  def run(self):
    for t in range(0, self.horizon):
      # Choose an arm to play.
      selected_arm = self.act(t)
       
      # Observe the reward of the selected arm.
      oracle_success_probability_of_selected_arm = self.oracle_success_probabilities[selected_arm]
      reward = np.random.binomial(1, oracle_success_probability_of_selected_arm)
       
      # Save the reward and the expected regret.
      self.reward_per_period[t] = reward
      self.regret_per_period[t] = np.max(self.oracle_success_probabilities) - oracle_success_probability_of_selected_arm
       
      # Cache the observation.
      self.batch_X.append(self.world.binary_feature_vector_per_arm[selected_arm])
      self.batch_y.append(reward)
       
      # Update the posteriors.
      if (t > 0 and t % self.posterior_updating_frequency == 0):
        self.update_weight_posteriors() 
         
      # Cache the observation.
      self.observations_cache[t, 0] = t
      self.observations_cache[t, 1] = selected_arm
      self.observations_cache[t, 2] = reward
       
        
  def update_weight_posteriors(self):
    batch_size = len(self.batch_X)
    # The outcomes and covariates of the batch.
    X = np.array(self.batch_X).astype(int)
    y = np.array(self.batch_y).astype(int)
     
    # Minimization objective.
    def objective(w): 
      p = np.empty(batch_size)
      for j in range(0, batch_size):
        p[j] = self.sigmoid(np.transpose(w).dot(X[j, :]))
       
      log_prior = - 0.5 * np.transpose(w - self.mu).dot(self.sigma_inverse).dot(w - self.mu)
      with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        log_likelihood = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))
       
      return -(log_prior + log_likelihood)
       
    # Minimization starting point.
    w0 = np.zeros(self.world.slot_count * self.world.level_count_per_slot)
    # Minimize and obtain w_MAP.
    result = minimize(objective, w0)
    w_MAP = result['x']
     
    self.mu = w_MAP.copy()
     
    sigma_inverse_update = np.zeros(self.sigma_inverse.shape)
    for j in range(0, batch_size):
      p_j = self.sigmoid(np.transpose(w_MAP).dot(X[j, :]))
      sigma_inverse_update += p_j * (1 - p_j) * np.outer(X[j, :], X[j, :])
    self.sigma_inverse += sigma_inverse_update
     
    # Clear the batch caches.
    self.clear_batch_caches()
   
  def clear_batch_caches(self):
    self.batch_X = []
    self.batch_y = []
   
  def sigmoid(self, x):
    try:
      return 1 / (1 + math.exp(-x))
    except OverflowError:
      return 0
  
  def act(self, t):
    pass
  
  
class LogisticRegressionTSAgent(LogisticRegressionAgent):
  def __init__(self, world, horizon, regularization_parameter):
    LogisticRegressionAgent.__init__(self, world, horizon, regularization_parameter)

  def act(self, t):
    # Sample the weights of the logistic function.
    mu = self.mu
    sigma = 1 / np.diag(self.sigma_inverse)
    sampled_weights = np.random.normal(loc=mu, scale=np.sqrt(sigma))
    
    level_per_slot = []
    for slot in range(self.world.slot_count):
      slice_start = slot * self.world.level_count_per_slot
      slice_end = (slot + 1) * self.world.level_count_per_slot
      level_per_slot.append(np.argmax(sampled_weights[slice_start:slice_end]))
    
    return self.world.slot_level_map_to_arm[tuple(level_per_slot)]
  
   

class LogisticRegressionUCBAgent(LogisticRegressionAgent):
  def __init__(self, world, horizon, regularization_parameter, UCB_tuning_parameter = 1):
    LogisticRegressionAgent.__init__(self, world, horizon, regularization_parameter)
    self.UCB_tuning_parameter = UCB_tuning_parameter
   
  def act(self, t):
    mu = self.mu
    sigma = np.diag(1 / np.diag(self.sigma_inverse))
     
    estimated_reward = np.zeros(self.world.arm_count)
    UCB_constant = 2 * (self.UCB_tuning_parameter**2) * math.log(t + 1)
    for arm in range(0, self.world.arm_count):
      x = self.world.binary_feature_vector_per_arm[arm]
      xT_sigma_x = np.transpose(x).dot(sigma).dot(x)
      estimated_reward[arm] = np.transpose(x).dot(mu) + np.sqrt(UCB_constant / xT_sigma_x) * xT_sigma_x
 
    if np.all(estimated_reward == estimated_reward[0]):
        return np.random.randint(self.world.arm_count)
    else:
        return np.argmax(estimated_reward)


In [ ]:
# Marginal Posterior Agents

class MarginalPosteriorTSAgent(MABAgent):
  def __init__(self, world, horizon):
    MABAgent.__init__(self, world, horizon)
     
    self.marginal_alphas = np.ones_like(self.world.marginals_per_slot_per_level)
    self.marginal_betas = np.ones_like(self.world.marginals_per_slot_per_level)
    
    self.posterior_updating_frequency = 20
    self.running_marginal_alphas = np.ones_like(self.world.marginals_per_slot_per_level)
    self.running_marginal_betas = np.ones_like(self.world.marginals_per_slot_per_level)
    
       
  def act(self, t):
    level_per_slot = []
    for slot in range(self.world.slot_count):
      level_per_slot.append(np.argmax(np.random.beta(self.marginal_alphas[slot, :], self.marginal_betas[slot, :])))
    
    return self.world.slot_level_map_to_arm[tuple(level_per_slot)]
   
  def update(self, selected_arm, reward, t):
    slot_level_map = self.world.slot_level_map_per_arm[selected_arm]
    for slot in range(self.world.slot_count):
      level = slot_level_map[slot]
      self.running_marginal_alphas[slot, level] += int(reward > 0)
      self.running_marginal_betas[slot, level] += int(reward == 0)
    if t > 0 and t % self.posterior_updating_frequency == 0:
      self.marginal_alphas = deepcopy(self.running_marginal_alphas)
      self.marginal_betas = deepcopy(self.running_marginal_betas)
      

class MarginalPosteriorUCBAgent(MABAgent):
  def __init__(self, world, horizon, UCB_tuning_parameter = 1):
    MABAgent.__init__(self, world, horizon)
    self.UCB_tuning_parameter = UCB_tuning_parameter
     
    self.marginal_alphas = np.ones_like(self.world.marginals_per_slot_per_level)
    self.marginal_betas = np.ones_like(self.world.marginals_per_slot_per_level)
       
  def act(self, t):
    UCB_constant = 2 * (self.UCB_tuning_parameter**2) * math.log(t + 1)
    
    level_per_slot = []
    for slot in range(self.world.slot_count):
      alphas = self.marginal_alphas[slot, :]
      betas = self.marginal_betas[slot, :]
      means, variances = self.mean_and_variance_of_beta(alphas, betas)
      level_per_slot.append(np.argmax(means + np.sqrt(UCB_constant * variances)))
    
    return self.world.slot_level_map_to_arm[tuple(level_per_slot)]

  def mean_and_variance_of_beta(self, alpha, beta):
    mean = alpha / (alpha + beta)
    variance = (alpha * beta) / ((alpha + beta)**2 * (alpha + beta + 1))
    return mean, variance
    
  def update(self, selected_arm, reward, t):
    slot_level_map = self.world.slot_level_map_per_arm[selected_arm]
    for slot in range(self.world.slot_count):
      level = slot_level_map[slot]
      self.marginal_alphas[slot, level] += int(reward > 0)
      self.marginal_betas[slot, level] += int(reward == 0)
